<h1>Содержание<span class="tocSkip"></span></h1>
<br>
<div class="toc">
</div>

# Лабораторная работа №3: Numba

**Задача:** ускорить алгоритмы градиентного спуска с помощью Numba.

**Источник данных:** набор данных взят с платформы [Kaggle](https://www.kaggle.com/datasets/rounakbanik/pokemon).

**Описание данных:** набор данных содержит информацию обо всех семи поколениях 802 Покемонов.

---

Для реализации поставленных задач из набора данных `datasets/pokemon.csv` будут взяты следующие столбцы:

* `defense` - признак объектов
* `attack` - целевой признак
* `name` - имя Покемона

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 1. Импорт библиотек

In [ ]:
import numba

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        Наверх к содержанию ↑
    </a>
</div>